In [1]:
from Ambient_Pipeline import Music_Gen_Pipeline
from LLMPromptGenerator import LLMPromptGenerator
from GenMusicFromPrompt import GenMusicFromPrompt
import torch
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [32]:
from transformers import pipeline

class DirectLLMPromptGenerator:
    def __init__(self, model_name="TheBloke/Llama-2-7b-Chat-GPTQ"):
        self.model_name = model_name
        self.hf_pipeline = pipeline('text-generation', model=model_name, device_map='auto')
        
    def generate_llm_prompt(self, text, prompt_format=None, few_shot=None):
        if prompt_format is None:
            prompt_format = """
            TASK: 
            From the provided text, generate a prompt for a music generation model such that the generated music encapsulates 
            the mood, tone, setting, and intensity of the text and can be used as background music for the text. Remember that
            the music generation model does not have the text as a reference, and will generate music based only on the prompt.
            Be sure to emphasize that the music should be a background track, and keep the prompt short.
            {}
            TEXT: 
            {} 
            PROMPT: 
            """
        if few_shot is True:
            few_shot = """
            Following are some example prompts to the music generation model. You can use them as inspiration to create your own prompt.

            EXAMPLE PROMPT 1:
            A grand orchestral arrangement with thunderous percussion, epic brass fanfares, and soaring strings, creating a cinematic atmosphere fit for a heroic battle.

            EXAMPLE PROMPT 2:
            drum and bass beat with intense percussions

            EXAMPLE PROMPT 3:
            Violins and synths that inspire awe at the finiteness of life and the universe.

            EXAMPLE PROMPT 4:
            A dynamic blend of hip-hop and orchestral elements, with sweeping strings and brass, evoking the vibrant energy of the city.
            """
        elif few_shot is None:
            few_shot = ""

        return prompt_format.format(few_shot, text)
    
    def generate_musicgen_prompt(self, text, prompt_format=None, few_shot=None):
        # Generate prompt for the LLM
        llm_prompt = self.generate_llm_prompt(text, prompt_format=prompt_format, few_shot=few_shot)
        
        #get the output (in json format) from the LLM
        llm_output = self.hf_pipeline(llm_prompt)
        generated_text = llm_output[0]['generated_text']
        
        return generated_text


In [33]:
extractor = DirectLLMPromptGenerator()
# generator = GenMusicFromPrompt(device=device)
# pipe = Music_Gen_Pipeline(extractor=extractor, generator=generator, device=device, verbose=True)

In [34]:
text_dir = "evaluation/excerpts/"
out_dir = "evaluation/generated_songs/"

In [35]:
test_audio_file = "evaluation/excerpts/rots.txt"

with open(test_audio_file) as f: 
    text = f.read()
text = ' '.join(text.split()[:60])  # limit to 60 words

prompt = extractor.generate_musicgen_prompt(text)
prompt_fs = extractor.generate_musicgen_prompt(text, few_shot=True)

print(prompt)
print(prompt_fs)

/home/cayjobla/PAGES/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1295: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



            TASK: 
            From the provided text, generate a prompt for a music generation model such that the generated music encapsulates 
            the mood, tone, setting, and intensity of the text and can be used as background music for the text. Remember that
            the music generation model does not have the text as a reference, and will generate music based only on the prompt.
            Be sure to emphasize that the music should be a background track, and keep the prompt short.
            
            TEXT: 
            A spray of lava from the river that melted one of the supports provided the final straw. A huge section of a collection arm broke away and fell into the lava, carrying the two Jedi with it. Still the fight continued, even as the collection tower sank slowly into the lava. And still, neither man could gain an advantage. But 
            PROMPT: 
            🔥🌋 Echoes of Lava 🔥🌋
            A haunting melody, born from the fiery depths of a river'

In [26]:
for i, filepath in enumerate(os.listdir(text_dir)):
    if not filepath.endswith(".txt"):   # Only grab text files
        continue

    with open(f"{text_dir}{filepath}") as f: 
        text = f.read()
    text = ' '.join(text.split()[:60])

    # Load the text section and generate the MusicGen prompt
    # text = pipe.text_to_sections(f"{text_dir}{filepath}")[0]        # Use to only take first section
    # text = pipe.load_txt(f"{text_dir}{filepath}")                   # Use to input entire excerpt
                                  # Only take first 60 words
    musicgen_prompt = extractor.generate_musicgen_prompt(text)    # Generate MusicGen prompt
    print(musicgen_prompt)
    break

    # Generate song
    pipe.generator.generate(musicgen_prompt, duration=10, prev_song_duration=0, song=None)
    pipe.generator.save_audio(f"{out_dir}{filepath.split('.')[0]}.wav")

/home/cayjobla/PAGES/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1295: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



            TASK: 
            From the provided text, generate a prompt for a music generation model such that the generated music encapsulates 
            the mood, tone, setting, and intensity of the text and can be used as background music for the text. Remember that
            the music generation model does not have the text as a reference, and will generate music based only on the prompt.
            Be sure to emphasize that the music should be a background track, and keep the prompt short.
            
            TEXT: 
            A spray of lava from the river that melted one of the supports provided the final straw. A huge section of a collection arm broke away and fell into the lava, carrying the two Jedi with it. Still the fight continued, even as the collection tower sank slowly into the lava. And still, neither man could gain an advantage. But 
            PROMPT: 
            🔥🌋 Echoes of Lava 🔥🌋
            A haunting melody, born from the fiery depths of a river'